In [2]:
# For testing if the llm is working or not

from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

model = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

result = model.invoke('What is the capital of India')

print(result.content)

c:\Users\dkum3\Desktop\Codes\try\ve\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The capital of India is **New Delhi**.


In [3]:
%pip install langchain openai faiss-cpu tiktoken
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # or HuggingFaceEmbeddings
from langchain.schema import Document
import os

In [6]:
# Load your .jsonl file
def load_jsonl(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

# Convert to LangChain Documents
def convert_to_documents(data):
    docs = []
    for item in data:
        metadata = {"url": item["url"], "title": item["title"]}
        docs.append(Document(page_content=item["content"], metadata=metadata))
    return docs

# Split each document into smaller chunks
def split_documents(documents, chunk_size=500, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "],
    )
    return splitter.split_documents(documents)

# Embed using GoogleAI or replace with HuggingFaceEmbeddings()
def create_embeddings_model():
    return GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Store to FAISS
def store_to_faiss(chunks, embeddings, save_path="faiss_index"):
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(save_path)
    return vectorstore

# Main pipeline
def build_rag_index(jsonl_path, output_path="faiss_index"):
    data = load_jsonl(jsonl_path)
    docs = convert_to_documents(data)
    chunks = split_documents(docs)
    embeddings = create_embeddings_model()
    store = store_to_faiss(chunks, embeddings, output_path)
    print(f"✅ Vector store saved to: {output_path}")

In [7]:
build_rag_index("openrewrite_scraper\cleaned_recipes.jsonl")

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\dkum3\AppData\Local\Temp\ipykernel_2128\337295160.py:1: SyntaxWarning: invalid escape sequence '\c'
  build_rag_index("openrewrite_scraper\cleaned_recipes.jsonl")


✅ Vector store saved to: faiss_index


In [8]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

db = FAISS.load_local(
    "faiss_index",
    GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    allow_dangerous_deserialization=True
)

In [9]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [10]:
from langchain.prompts import PromptTemplate

template = """
You are an expert OpenRewrite assistant helping developers perform Java code migrations.

You are given some documentation chunks as context. Follow these rules:

1. If the user's request matches a known recipe (described in the context), suggest the recipe name(s) and explain briefly how to use it.
2. If no exact recipe exists, check if the request could be fulfilled by creating a custom recipe.
   - If yes, explain how to create a custom recipe using `rewrite.yml`, Java DSL, or by extending OpenRewrite base classes.
   - Use examples from the context if available.
3. If neither is possible, say:
   "There is no known recipe or migration method available for this case."

Always cite the source URL(s) if you use any documentation.

--------------------
Context:
{context}

Question:
{question}
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [12]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)

In [13]:
query = "Give me recipe to convert spring 2 to spring 3"

response = qa_chain.invoke(query)
print(response['result'])

There is no single recipe to convert Spring 2 to Spring 3 directly. However, you can use a combination of recipes to achieve this migration. Since Spring Boot simplifies Spring application development, I assume you are using Spring Boot. Here's a possible approach:

1.  **Migrate Spring Boot properties:** Use the `org.openrewrite.java.spring.boot3.SpringBootProperties_3_0` recipe to migrate Spring Boot properties to 3.0.  Then use `org.openrewrite.java.spring.boot3.SpringBootProperties_3_1` to migrate properties to 3.1 and `org.openrewrite.java.spring.boot3.SpringBootProperties_3_2` to migrate properties to 3.2. ([https://docs.openrewrite.org/reference/recipe/org.openrewrite.java.spring.boot3.SpringBootProperties_3_2](https://docs.openrewrite.org/reference/recipe/org.openrewrite.java.spring.boot3.SpringBootProperties_3_2))
2.  **Upgrade Spring Boot version:** Use the `org.openrewrite.java.spring.boot3.UpgradeSpringBoot_3_2` recipe to migrate your application to the latest Spring Boot 3

In [14]:
print(response)

{'query': 'Give me recipe to convert spring 2 to spring 3', 'result': "There is no single recipe to convert Spring 2 to Spring 3 directly. However, you can use a combination of recipes to achieve this migration. Since Spring Boot simplifies Spring application development, I assume you are using Spring Boot. Here's a possible approach:\n\n1.  **Migrate Spring Boot properties:** Use the `org.openrewrite.java.spring.boot3.SpringBootProperties_3_0` recipe to migrate Spring Boot properties to 3.0.  Then use `org.openrewrite.java.spring.boot3.SpringBootProperties_3_1` to migrate properties to 3.1 and `org.openrewrite.java.spring.boot3.SpringBootProperties_3_2` to migrate properties to 3.2. ([https://docs.openrewrite.org/reference/recipe/org.openrewrite.java.spring.boot3.SpringBootProperties_3_2](https://docs.openrewrite.org/reference/recipe/org.openrewrite.java.spring.boot3.SpringBootProperties_3_2))\n2.  **Upgrade Spring Boot version:** Use the `org.openrewrite.java.spring.boot3.UpgradeSpri

In [15]:
# print(response["source_documents"])
for doc in response["source_documents"]:
    # print(doc)
    print("From:", doc.metadata["url"])

From: https://docs.openrewrite.org/recipes/java/spring/boot3/springbootproperties_3_2
From: https://docs.openrewrite.org/recipes/java/spring/boot3/upgradespringboot_3_2
From: https://docs.openrewrite.org/recipes/java/spring/boot2/upgradespringboot_2_3
From: https://docs.openrewrite.org/recipes/java/spring/boot3
